In [ ]:
import glob
import os
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from PIL import Image
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

    

In [ ]:
imgs = glob.glob("./img_nihai/*.png")


width = 125
height = 50

X = []
Y = []

for img in imgs: #images icindekileri tek tek alıyoruz
     
    filename = os.path.basename(img)  #dosya adını alıyoruz

    label = filename.split("_")[0] #dosya adını _ ile ayırıyoruz ve ilk kısmını alıyoruz

    im = np.array(Image.open(img).convert("L").resize((width, height))) #resmi açıyoruz ve boyutlandırıyoruz    
    im = im / 255 #normalizasyon yapıyoruz
    X.append(im) #resmi X listesine ekliyoruz
    Y.append(label) #labeli Y listesine ekliyoruz
     
X = np.array(X) #listeyi numpy arrayine çeviriyoruz
X = X.reshape(X.shape[0], width, height, 1) #resimlerin boyutunu düzenliyoruz

# sns.countplot(Y)

def onehot_labels(values):
    label_encoder = LabelEncoder() #label encoder oluşturuyoruz

    integer_encoded = label_encoder.fit_transform(values) #label encoder ile labeli integera çeviriyoruz

    onehot_encoder = OneHotEncoder(sparse = False) #onehot encoder oluşturuyoruz

    integer_encoded = integer_encoded.reshape(len(integer_encoded),1) #integer_encodedi düzenliyoruz

    onehot_encoded = onehot_encoder.fit_transform(integer_encoded) #integer_encodedi onehot_encodede çeviriyoruz
    return onehot_encoded

Y = onehot_labels(Y) #labeli onehot_encodede çeviriyoruz
train_X, test_X, train_y, test_y = train_test_split(X, Y , test_size = 0.25, random_state = 2)   #veriyi train ve test olarak ayırıyoruz  

# cnn model 
model = Sequential() #modeli oluşturuyoruz  

model.add(Conv2D(32, kernel_size = (3,3), activation = "relu", input_shape = (width, height, 1))) #erisinim katmanı ekliyoruz

model.add(Conv2D(64, kernel_size = (3,3), activation = "relu")) #erisinim katmanı ekliyoruz

model.add(MaxPooling2D(pool_size = (2,2))) #piksel havuzlama katmanı ekliyoruz

model.add(Dropout(0.25)) #seyreltme katmanı ekliyoruz

model.add(Flatten()) #düzleştirme katmanı ekliyoruz

model.add(Dense(128, activation = "relu")) #yoğun katman ekliyoruz
model.add(Dropout(0.4)) #seyreltme katmanı ekliyoruz
model.add(Dense(3, activation = "softmax")) #yoğun katman ekliyoruz


model.compile(loss = "categorical_crossentropy", optimizer = "Adam", metrics = ["accuracy"]) #modeli derliyoruz

model.fit(train_X, train_y, epochs = 35, batch_size = 64) #modeli eğitiyoruz

score_train = model.evaluate(train_X, train_y) #eğitim doğruluğunu hesaplıyoruz
print("Eğitim doğruluğu: %",score_train[1]*100)    
    
score_test = model.evaluate(test_X, test_y) #test doğruluğunu hesaplıyoruz
print("Test doğruluğu: %",score_test[1]*100)      
    
 
open("model.json","w").write(model.to_json()) #modeli json formatında kaydediyoruz
model.save_weights("trex_weights.weights.h5")   